In [35]:
import sys
from datetime import datetime
from os.path import join
from warnings import warn
from itertools import chain

import numpy as np
import pandas as pd
import scipy as sp

from utils import *

In [36]:
dir_arg = sys.argv[1]
if dir_arg == '-f':
    file_dir = join('..', 'dataset', '11')
else:
    file_dir = join('..', 'dataset',  dir_arg)

In [37]:
train_df = pd.read_pickle(join(file_dir, 'base_feauture.pkl'))

sample = pd.read_pickle(join(file_dir, 'uid.pkl'))

now_date = train_df.orderdate.max().date()
print(datetime.now(), now_date)

uid_shape, hotelid_shape, basicroomid_shape, roomid_shape = print_shape(
    train_df, ['uid', 'hotelid', 'basicroomid', 'roomid'])

2017-08-01 21:01:47.716732 2017-06-11
2017-08-01 21:01:47.716965
--------------------
uid uniuqe shape 3552
hotelid uniuqe shape 3194
basicroomid uniuqe shape 19542
roomid uniuqe shape 121802
--------------------


In [38]:
feature_path = join(file_dir, 'user_feature.pkl')
print(datetime.now(), 'begin', feature_path)

2017-08-01 21:01:48.415813 begin ../dataset/11/user_feature.pkl


In [39]:
user_oreder_type = [x for x in train_df.columns if x.startswith('ordertype')]
user_orderbehavior = [x for x in train_df.columns if x.startswith('orderbehavior')]
user_lastord = [x for x in train_df.columns if x.endswith('lastord')]
user_feature_cols = [x for x in train_df.columns if x.startswith('user')]

## 添加基本特征

In [40]:
sample = extract_feature_count('uid', 'hotel_roomid', train_df, sample)

In [41]:
def extract_user_feature_is_equal(t, train_df, sample):
    name = '{}_is_equal'.format(t)
    lastord_name = t + '_lastord'
    train_df[name] = np.nan
    train_df.loc[(train_df[lastord_name] == train_df[t]), name] = 1
    sample = extract_feature_count('uid', name, train_df, sample)
    sample = press_date(sample, ['uid' + '__' + name + '_count'])
    return sample

In [42]:
for i in range(2, 9):
    t = 'roomservice_%d' % i
    if i != 7:
        sample = extract_user_feature_is_equal(t, train_df, sample)

In [43]:
for i in range(2, 5):
    t = 'roomtag_%d' % i
    sample = extract_user_feature_is_equal(t, train_df, sample)

In [44]:
for c in ['rank', 'star']:
    sample = extract_user_feature_is_equal(c, train_df, sample)

In [45]:
user_cols = list(chain(user_oreder_type, user_orderbehavior, user_feature_cols))

In [46]:
add_cols = ['hotel_minprice_lastord', 'basic_minprice_lastord', 'star_lastord'] + user_cols

In [50]:
not_use = []

In [51]:
for col in add_cols:
    if col not in not_use:
        sample = add_column(train_df, sample, 'uid', col)

/home/zhanglun/notebook/room/code/utils.py:30: UserWarning: column  uid_ordertype_3_ratio is  may be error when meet percent max:0.1363525390625
  warn('column  {} is  may be error when meet percent max:{}'.format(c, c_max))
/home/zhanglun/notebook/room/code/utils.py:30: UserWarning: column  uid_user_roomservice_4_1ratio is  may be error when meet percent max:0.5
  warn('column  {} is  may be error when meet percent max:{}'.format(c, c_max))


In [15]:
# get_corr(train_df, sample, 'uid')

In [16]:
# corr = _

# cor = abs(corr['orderlabel'])

# [x for x in cor.loc[np.isnan(cor)].index]

# [x[4:] for x in cor.loc[cor<0.01].index]  + [x[4:] for x in cor.loc[np.isnan(cor)].index]

## 基本交叉特征

In [56]:
sample.columns

Index(['uid', 'uid__hotel_roomid_count', 'uid__roomservice_2_is_equal_count',
       'uid__roomservice_3_is_equal_count',
       'uid__roomservice_4_is_equal_count',
       'uid__roomservice_5_is_equal_count',
       'uid__roomservice_6_is_equal_count',
       'uid__roomservice_8_is_equal_count', 'uid__roomtag_2_is_equal_count',
       'uid__roomtag_3_is_equal_count',
       ...
       'uid_user_minprice_3month', 'uid_user_maxprice_3month',
       'uid_user_roomservice_3_123ratio_3month',
       'uid_user_roomservice_7_1ratio_3month',
       'uid_user_roomservice_7_0ratio_3month',
       'uid_user_roomservice_4_5ratio_3month',
       'uid_user_roomservice_4_4ratio_3month',
       'uid_user_roomservice_4_2ratio_3month',
       'uid_user_roomservice_4_3ratio_3month',
       'uid_user_roomservice_4_0ratio_3month'],
      dtype='object', length=114)

In [66]:
press_columns = ['uid_user_roomservice_8_2ratio', 'uid_user_roomservice_4_1ratio_3month',
   'uid_user_roomservice_4_1ratio_1month', 'uid_user_roomservice_4_1ratio_1week',
                'uid_user_roomservice_2_0ratio', 'uid_user_roomservice_3_0ratio',
                'uid_user_roomservice_5_0ratio', 'uid_user_roomservice_7_1ratio',
                'uid_user_roomservice_2_max', 'uid_user_roomservice_3_max',
                'uid_user_roomservice_5_max', 'uid_user_roomservice_7_max',
                'uid_user_roomservice_4_max', 'vuser_roomservice_6_max',
                'uid_user_roomservice_8_max', 'uid_user_roomservice_4_max_1week', 
                'uid_user_roomservice_4_max_1month',
                'uid_user_roomservice_4_max_3month']

In [67]:
# sample["user_roomservice_8_345ratio"]=sample["user_roomservice_5_345ratio"]
# del sample["user_roomservice_5_345ratio"]
sample[
    "uid_user_roomservice_8_2ratio"] = 1 - sample["uid_user_roomservice_8_345ratio"] - sample["uid_user_roomservice_8_1ratio"]
sample[
    "uid_user_roomservice_4_1ratio_3month"] = 1 - sample["uid_user_roomservice_4_0ratio_3month"] - sample["uid_user_roomservice_4_2ratio_3month"] - sample["uid_user_roomservice_4_3ratio_3month"] - sample["uid_user_roomservice_4_4ratio_3month"] - sample["uid_user_roomservice_4_5ratio_3month"]
sample[
    "uid_user_roomservice_4_1ratio_1month"] = 1 - sample["uid_user_roomservice_4_0ratio_1month"] - sample["uid_user_roomservice_4_2ratio_1month"] - sample["uid_user_roomservice_4_3ratio_1month"] - sample["uid_user_roomservice_4_4ratio_1month"] - sample["uid_user_roomservice_4_5ratio_1month"]
sample[
    "uid_user_roomservice_4_1ratio_1week"] = 1 - sample["uid_user_roomservice_4_0ratio_1week"] - sample["uid_user_roomservice_4_2ratio_1week"] - sample["uid_user_roomservice_4_3ratio_1week"] - sample["uid_user_roomservice_4_4ratio_1week"] - sample["uid_user_roomservice_4_5ratio_1week"]
sample["uid_user_roomservice_2_0ratio"] = 1 - sample["uid_user_roomservice_2_1ratio"]
sample["uid_user_roomservice_3_0ratio"] = 1 - sample["uid_user_roomservice_3_123ratio"]
sample["uid_user_roomservice_5_0ratio"] = 1 - sample["uid_user_roomservice_5_1ratio"]
sample["uid_user_roomservice_7_1ratio"] = 1 - sample["uid_user_roomservice_7_0ratio"]
sample["uid_user_roomservice_2_max"] = np.argmax(
    sample[["uid_user_roomservice_2_%sratio" % i for i in range(2)]].values,
    axis=1)
sample["uid_user_roomservice_3_max"] = np.argmax(
    sample[["uid_user_roomservice_3_%sratio" % i for i in [0, 123]]].values,
    axis=1)
sample["uid_user_roomservice_5_max"] = np.argmax(
    sample[["uid_user_roomservice_5_%sratio" % i for i in range(2)]].values,
    axis=1)
sample["uid_user_roomservice_7_max"] = np.argmax(
    sample[["uid_user_roomservice_7_%sratio" % i for i in range(2)]].values,
    axis=1)
sample["uid_user_roomservice_4_max"] = np.argmax(
    sample[["uid_user_roomservice_4_%sratio" % i for i in range(6)]].values,
    axis=1)
sample["vuser_roomservice_6_max"] = np.argmax(
    sample[["uid_user_roomservice_6_%sratio" % i for i in range(3)]].values,
    axis=1)
sample["uid_user_roomservice_8_max"] = np.argmax(
    sample[["uid_user_roomservice_8_%sratio" % i for i in [1, 2, 345]]].values,
    axis=1)
sample["uid_user_roomservice_4_max_1week"] = np.argmax(
    sample[["uid_user_roomservice_4_%sratio_1month" % i for i in range(6)]].values,
    axis=1)
sample["uid_user_roomservice_4_max_1month"] = np.argmax(
    sample[["uid_user_roomservice_4_%sratio_1month" % i for i in range(6)]].values,
    axis=1)
sample["uid_user_roomservice_4_max_3month"] = np.argmax(
    sample[["uid_user_roomservice_4_%sratio_3month" % i for i in range(6)]].values,
    axis=1)

In [68]:
sample = press_date(sample, press_columns)

## 历史价格统计特征 

In [9]:
name_fmt = '{}_diff_{}'.format('uid', '{}')

price_diff_name = name_fmt.format('price_last_lastord')
hotel_minprice_diff_name = name_fmt.format('hotel_minprice_lastord')
basic_minprice_diff_name = name_fmt.format('basic_minprice_lastord')

In [10]:
train_df[price_diff_name] = train_df['price_deduct'] - train_df['price_last_lastord']
train_df[hotel_minprice_diff_name] = train_df['price_deduct'] - train_df['hotel_minprice_lastord']
train_df[basic_minprice_diff_name] = train_df['price_deduct'] - train_df['basic_minprice_lastord']

In [ ]:
price_describe = ['mean', 'median']

In [11]:
sample = extract_value_describe_feature('uid', price_diff_name, train_df, sample, price_describe)
sample = extract_value_describe_feature('uid', hotel_minprice_diff_name, train_df, sample, price_describe)
sample = extract_value_describe_feature('uid', basic_minprice_diff_name, train_df, sample, price_describe)

In [16]:
# get_corr(train_df, sample, 'uid')

,orderlabel,uid__uid_diff_price_last_lastord_count,uid__uid_diff_price_last_lastord_mean,uid__uid_diff_price_last_lastord_std,uid__uid_diff_price_last_lastord_min,uid__uid_diff_price_last_lastord_25,uid__uid_diff_price_last_lastord_50,uid__uid_diff_price_last_lastord_75,uid__uid_diff_price_last_lastord_max,uid__uid_diff_hotel_minprice_lastord_count,...,uid__uid_diff_hotel_minprice_lastord_max,uid__uid_diff_basic_minprice_lastord_count,uid__uid_diff_basic_minprice_lastord_mean,uid__uid_diff_basic_minprice_lastord_std,uid__uid_diff_basic_minprice_lastord_min,uid__uid_diff_basic_minprice_lastord_25,uid__uid_diff_basic_minprice_lastord_50,uid__uid_diff_basic_minprice_lastord_75,uid__uid_diff_basic_minprice_lastord_max,uid__hotel_roomid_count
orderlabel,1.000000,-0.082468,-0.027297,-0.019170,-0.006431,-0.017837,-0.025349,-0.032527,-0.020614,-0.082468,...,-0.020758,-0.082468,-0.027825,-0.019170,-0.007673,-0.018998,-0.026308,-0.033209,-0.020662,-0.092254
uid__uid_diff_price_last_lastord_count,-0.082468,1.000000,0.287712,0.183210,0.108050,0.225623,0.303479,0.358626,0.210281,1.000000,...,0.211220,1.000000,0.292979,0.183210,0.120997,0.237552,0.313198,0.365457,0.210759,0.910141
uid__uid_diff_price_last_lastord_mean,-0.027297,0.287712,1.000000,0.796070,0.636920,0.742495,0.810900,0.856718,0.783407,0.287712,...,0.774295,0.287712,0.999195,0.796070,0.645171,0.747981,0.813461,0.857065,0.783188,0.287712
uid__uid_diff_price_last_lastord_std,-0.019170,0.183210,0.796070,1.000000,0.132056,0.234659,0.325356,0.430358,0.977654,0.183210,...,0.978502,0.183210,0.798789,1.000000,0.141157,0.242947,0.331959,0.434760,0.977695,0.183210
uid__uid_diff_price_last_lastord_min,-0.006431,0.108050,0.636920,0.132056,1.000000,0.969501,0.900895,0.754034,0.179880,0.108050,...,0.158218,0.108050,0.628860,0.132056,0.996609,0.962180,0.891422,0.745002,0.179105,0.108050
uid__uid_diff_price_last_lastord_25,-0.017837,0.225623,0.742495,0.234659,0.969501,1.000000,0.973238,0.866761,0.279373,0.225623,...,0.260349,0.225623,0.736647,0.234659,0.970605,0.997232,0.967678,0.860579,0.278794,0.225623
uid__uid_diff_price_last_lastord_50,-0.025349,0.303479,0.810900,0.325356,0.900895,0.973238,1.000000,0.940006,0.364421,0.303479,...,0.348563,0.303479,0.807122,0.325356,0.905658,0.974302,0.997945,0.936450,0.364025,0.303479
uid__uid_diff_price_last_lastord_75,-0.032527,0.358626,0.856718,0.430358,0.754034,0.866761,0.940006,1.000000,0.450263,0.358626,...,0.439372,0.358626,0.854808,0.430358,0.761046,0.870707,0.940870,0.998792,0.450026,0.358626
uid__uid_diff_price_last_lastord_max,-0.020614,0.210281,0.783407,0.977654,0.179880,0.279373,0.364421,0.450263,1.000000,0.210281,...,0.999525,0.210281,0.785721,0.977654,0.188675,0.287172,0.370451,0.454149,0.999992,0.210281
uid__uid_diff_hotel_minprice_lastord_count,-0.082468,1.000000,0.287712,0.183210,0.108050,0.225623,0.303479,0.358626,0.210281,1.000000,...,0.211220,1.000000,0.292979,0.183210,0.120997,0.237552,0.313198,0.365457,0.210759,0.910141


### 历史订单间隔统计特征 

In [38]:
span_name, t = '{}_ordspan'.format('uid'), 'uid'

In [43]:
# train_df[span_name] = (now_date - train_df.orderdate_lastord).dt.days

# sample = extract_value_describe_feature(t, span_name, train_df, sample, ['max', 'min', 'mean'])

In [42]:
# get_corr(train_df, sample, 'uid')

,orderlabel,uid__uid_diff_price_last_lastord_count,uid__uid_diff_price_last_lastord_mean,uid__uid_diff_price_last_lastord_std,uid__uid_diff_price_last_lastord_min,uid__uid_diff_price_last_lastord_25,uid__uid_diff_price_last_lastord_50,uid__uid_diff_price_last_lastord_75,uid__uid_diff_price_last_lastord_max,uid__uid_diff_hotel_minprice_lastord_count,...,uid__uid_diff_basic_minprice_lastord_mean,uid__uid_diff_basic_minprice_lastord_std,uid__uid_diff_basic_minprice_lastord_min,uid__uid_diff_basic_minprice_lastord_25,uid__uid_diff_basic_minprice_lastord_50,uid__uid_diff_basic_minprice_lastord_75,uid__uid_diff_basic_minprice_lastord_max,uid__uid_ordspan_max,uid__uid_ordspan_min,uid__uid_ordspan_mean
orderlabel,1.000000,-0.082468,-0.027297,-0.019170,-0.006431,-0.017837,-0.025349,-0.032527,-0.020614,-0.082468,...,-0.027825,-0.019170,-0.007673,-0.018998,-0.026308,-0.033209,-0.020662,0.003112,0.003112,0.003112
uid__uid_diff_price_last_lastord_count,-0.082468,1.000000,0.287712,0.183210,0.108050,0.225623,0.303479,0.358626,0.210281,1.000000,...,0.292979,0.183210,0.120997,0.237552,0.313198,0.365457,0.210759,0.016251,0.016251,0.016251
uid__uid_diff_price_last_lastord_mean,-0.027297,0.287712,1.000000,0.796070,0.636920,0.742495,0.810900,0.856718,0.783407,0.287712,...,0.999195,0.796070,0.645171,0.747981,0.813461,0.857065,0.783188,0.013835,0.013835,0.013835
uid__uid_diff_price_last_lastord_std,-0.019170,0.183210,0.796070,1.000000,0.132056,0.234659,0.325356,0.430358,0.977654,0.183210,...,0.798789,1.000000,0.141157,0.242947,0.331959,0.434760,0.977695,0.012476,0.012476,0.012476
uid__uid_diff_price_last_lastord_min,-0.006431,0.108050,0.636920,0.132056,1.000000,0.969501,0.900895,0.754034,0.179880,0.108050,...,0.628860,0.132056,0.996609,0.962180,0.891422,0.745002,0.179105,0.014939,0.014939,0.014939
uid__uid_diff_price_last_lastord_25,-0.017837,0.225623,0.742495,0.234659,0.969501,1.000000,0.973238,0.866761,0.279373,0.225623,...,0.736647,0.234659,0.970605,0.997232,0.967678,0.860579,0.278794,0.022829,0.022829,0.022829
uid__uid_diff_price_last_lastord_50,-0.025349,0.303479,0.810900,0.325356,0.900895,0.973238,1.000000,0.940006,0.364421,0.303479,...,0.807122,0.325356,0.905658,0.974302,0.997945,0.936450,0.364025,0.026856,0.026856,0.026856
uid__uid_diff_price_last_lastord_75,-0.032527,0.358626,0.856718,0.430358,0.754034,0.866761,0.940006,1.000000,0.450263,0.358626,...,0.854808,0.430358,0.761046,0.870707,0.940870,0.998792,0.450026,0.018517,0.018517,0.018517
uid__uid_diff_price_last_lastord_max,-0.020614,0.210281,0.783407,0.977654,0.179880,0.279373,0.364421,0.450263,1.000000,0.210281,...,0.785721,0.977654,0.188675,0.287172,0.370451,0.454149,0.999992,0.019975,0.019975,0.019975
uid__uid_diff_hotel_minprice_lastord_count,-0.082468,1.000000,0.287712,0.183210,0.108050,0.225623,0.303479,0.358626,0.210281,1.000000,...,0.292979,0.183210,0.120997,0.237552,0.313198,0.365457,0.210759,0.016251,0.016251,0.016251


In [28]:
train_df.shape

(130801, 154)

In [ ]:
sample.to_pickle(feature_path)

print(datetime.now(), 'save to', feature_path)